In [ ]:
import time as time
import pandas as pd


from selenium import webdriver 
from selenium.webdriver.support.ui import Select

In [ ]:
# Functions scrapes the page for postal codes
def getPostalCodes():
   
    # helped functions that removes any postal codes that are not being used by Canada Post
    def removeUnassignedPostalCode(x):
        try:
            checkString = x.find_element_by_tag_name('i').text
        except:
            return True

        if "Not" in checkString:
            return False 

    # Loads the urban table postal codes     
    urbanTable = browser.find_elements_by_tag_name('table')[0]
    
    # Loads the rural table postal codes 
    ruralTable = browser.find_elements_by_tag_name('table')[1]

    # Removes unused postal codes html elements from the list
    urbanList = list(filter(removeUnassignedPostalCode, urbanTable.find_elements_by_tag_name('td')))
    
    # Gets the text from the element that has the postal code
    urbanList = list(map(lambda x : x.find_element_by_tag_name('b').text, urbanList))
    
    # Removes unused postal codes html elements from the list
    ruralList = list(filter(removeUnassignedPostalCode, ruralTable.find_elements_by_tag_name('td')))
    
    # Gets the text from the element that has the postal code
    ruralList = list(map(lambda x : x.find_element_by_tag_name('b').text, ruralList))
    
    return (ruralList+urbanList)

# gets postal code for Toronto; its a different layout so needs its own function
def getPostalCodesforM():
    
    # helped functions that removes any postal codes that are not being used by Canada Post
    def removeUnassignedPostalCode(x):
        checkString = x.text
        if "Not" in checkString:
            return False 
        return True
   
    # Grabs the table with the codes
    torontoTable = browser.find_element_by_tag_name('table')
    
    # Gets the rows with the code but also removes the header row 
    postalCodeList = torontoTable.find_elements_by_tag_name('tr')[1:]
    
    # Removes unused postal codes
    postalCodeList = list(filter(removeUnassignedPostalCode, postalCodeList))
    
    # Grabs the text from the elements with the postal codes
    postalCodeList = list(map(lambda x : x.find_element_by_tag_name('td').text, postalCodeList))
    
    return postalCodeList

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_'

# All the first letter postal codes used in Ontario
postalLetters = {'P', 'K', 'M', 'L', 'M', 'N'}

# Create an empty array that will house the postal codes
postalCodes = list()

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument('-incognito')

browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

In [ ]:
# For each letter go to their wikipage and scrape the tables 
for letter in postalLetters:
    
    # Go to the wikipage with the codes
    browser.get(url+letter)
    time.sleep(1)
    
    # if its the Letter M (toronto) then use the special function for toronto else use the regular one
    if letter == 'M':
        postalCodes += getPostalCodesforM()
    else:
        postalCodes += getPostalCodes()

In [ ]:
# Make a dataframe object out of the postalcode list for easier maniuplation
postalCSV = pd.DataFrame(columns={'Postal Code'}, data=postalCodes)

# Drop any duplicates but keep the first occurence of a postal code
postalCSV.drop_duplicates(keep='first', inplace=True)

# Save it
postalCSV.to_csv('./csv/ON Postal Codes.csv', index=False)